# Seq2Seq Prefix-Tuning for C++

This is the main notebook for prefix-tuning *codet5p-220m-bimodal seq2seq* on C++ programming language dataset.

## Clone the repository

In [1]:
!git clone https://github.com/leiluk1/CodeSearcher.git && cd CodeSearcher/ && git checkout dev/embeddings

Cloning into 'CodeSearcher'...
remote: Enumerating objects: 252, done.
remote: Counting objects: 100% (252/252), done.
remote: Compressing objects: 100% (164/164), done.
remote: Total 252 (delta 138), reused 188 (delta 83), pack-reused 0
Receiving objects: 100% (252/252), 7.17 MiB | 5.37 MiB/s, done.
Resolving deltas: 100% (138/138), done.
branch 'dev/embeddings' set up to track 'origin/dev/embeddings'.
Switched to a new branch 'dev/embeddings'


## Set up the required dependencies

In [2]:
!pip install dataprep gdown py7zr transformers peft evaluate rouge_score fire loguru --quiet

In [3]:
import wandb
wandb.login(key='b3b8dfa596c6a59225a25dd6be96a4325a7eaed5')

wandb: Currently logged in as: ley-khaertdinova. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/buiniy_yarik/.netrc


True

In [4]:
!mkdir -p CodeSearcher/output
!mkdir -p CodeSearcher/data/raw
!gdown 1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
!unzip -q -d CodeSearcher/data/raw ./XLCoST_data.zip

Downloading...
From (uriginal): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6
From (redirected): https://drive.google.com/uc?id=1tZfsYQgWmc2gG340ru5VbrZ5aLIZ41_6&confirm=t&uuid=c839de6e-c82c-4afb-873d-4e964765f824
To: /home/buiniy_yarik/DataspellProjects/Karina_PMLDL/XLCoST_data.zip
100%|████████████████████████████████████████| 298M/298M [00:08<00:00, 34.4MB/s]


## Perform prefix-tuning 

For more details, please check the code in `src/models/train.py` in our repository.

In [5]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/train.py seq2seq prefix \
    --output_dir="./output" \
    --epochs=10 \
    --num_virtual_tokens=10 \
    --language="C++" 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2023-11-24 14:03:07.667 | INFO     | __main__:_setup_peft_model:28 - trainable params: 184,320 || all params: 223,267,074 || trainable%: 0.08255583624480159
2023-11-24 14:03:07.812 | INFO     | src.datasets.XLCoST.make_dataset:_load_search_dataframe:84 - Loading dataframe from data/raw/XLCoST_data/retrieval/nl2code_search/snippet_level/C++/train.jsonl
/home/buiniy_yarik/DataspellProjects/Karina_PMLDL/CodeSearcher/src/datasets/XLCoST/make_dataset.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_trunc['code_tokens'] = dataframe_trunc['code_tokens'].apply(_code_tokens_to_str)
2023-11-24 14:03:08.529 | INFO     | src.dat

## Save checkpoint

In [6]:
!zip -r seq2seq_prefix_c.zip CodeSearcher/output

  adding: CodeSearcher/output/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-3500/ (stored 0%)
  adding: CodeSearcher/output/checkpoint-3500/README.md (deflated 65%)
  adding: CodeSearcher/output/checkpoint-3500/adapter_model.safetensors (deflated 7%)
  adding: CodeSearcher/output/checkpoint-3500/adapter_config.json (deflated 37%)
  adding: CodeSearcher/output/checkpoint-3500/tokenizer_config.json (deflated 95%)
  adding: CodeSearcher/output/checkpoint-3500/special_tokens_map.json (deflated 82%)
  adding: CodeSearcher/output/checkpoint-3500/added_tokens.json (deflated 37%)
  adding: CodeSearcher/output/checkpoint-3500/vocab.json (deflated 59%)
  adding: CodeSearcher/output/checkpoint-3500/merges.txt (deflated 54%)
  adding: CodeSearcher/output/checkpoint-3500/tokenizer.json (deflated 72%)
  adding: CodeSearcher/output/checkpoint-3500/training_args.bin (deflated 49%)
  adding: CodeSearcher/output/checkpoint-3500/optimizer.pt (deflated 53%)
  adding: CodeSearcher/output/checkpoint

## Evaluation

In this step, test the model using *Mean Reciprocal Rank (MRR)* as the evaluation metric. 

For more details, please refer to `src/models/evaluation.py` in the repository.

In [7]:
!export DATASETS_VERBOSITY=error
!cd CodeSearcher/ && export PYTHONPATH=. && python src/models/evaluation.py \
    --tuned_ckpt_path="output/best_ckpt" \
    --num_virtual_tokens=10 \
    --language="C++"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2023-11-24 15:40:05.018 | INFO     | src.datasets.XLCoST.make_dataset:_load_search_dataframe:84 - Loading dataframe from data/raw/XLCoST_data/retrieval/nl2code_search/snippet_level/C++/train.jsonl
/home/buiniy_yarik/DataspellProjects/Karina_PMLDL/CodeSearcher/src/datasets/XLCoST/make_dataset.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_trunc['code_tokens'] = dataframe_trunc['code_tokens'].apply(_code_tokens_to_str)
2023-11-24 15:40:05.682 | INFO     | src.datasets.XLCoST.make_dataset:__init__:64 - XLCoST C++ train generation=False dataset length: 62839
2023-11-24 15:40:05.682 | INFO     | src.datasets.XLCoST.make

# Results:

- Test MRR for C++: **0.13243804**;
- Epochs: **10**;
- Trainable params: 184,320 || all params: 223,267,074 || trainable%: 0.08255583624480159.

You can also check the validation plots in [wanb report](https://api.wandb.ai/links/ley-khaertdinova/13lvn64p).